In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import bs4
%matplotlib inline
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import statsmodels.api as sm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
import sys
reload(sys)
sys.setdefaultencoding('utf8')
#matplotlib.style.use('ggplot') 

#!pip install imdbpie

In [3]:
# from imdbpie import Imdb
# imdb = Imdb()
# imdb = Imdb(anonymize=True) # to proxy requests

# # Creating an instance with caching enabled
# # Note that the cached responses expire every 2 hours or so.
# # The API response itself dictates the expiry time)
# imdb = Imdb(cache=True)
# df = pd.DataFrame(imdb.top_250())
# df.head()

In [9]:
#scraping list of worst movies from wikipedia 

url = 'https://en.wikipedia.org/wiki/List_of_films_considered_the_worst'
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")


def worst(soupy):
    wor = soupy.findAll(class_='mw-headline')

    worst = []
    for i in wor:
        a = ['1930s','1940s','1950s','1960s','1970s','1980s','1990s','2000s','2010s','See also','References','External links']
        if i.text.strip() not in a:
            try:
                worst.append(i.text.strip())
            except:
                worst.append(np.nan)
    return worst

worstmovies = worst(soup)

In [17]:
#removing year from the movie title

for i in range(len(worstmovies)):
    worstmovies[i] = worstmovies[i][:worstmovies[i].find("(")]

In [18]:
#scraping 1000 best movies of all time from NYTimes

url = 'http://www.nytimes.com/ref/movies/1000best.html?pagewanted=print'
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

def nytimesmovies(soupy):
    nyt = soupy.findAll('td')

    movies = []
    for i in nyt:
        try:
            movies.append(i.text.strip())
        except:
            movies.append(np.nan)

    return movies


moremovies = nytimesmovies(soup)

moremovies2 = []
for i in moremovies:
    if i != "":
        a = i.find('(')
        i = i[:a-1]
        str(i)
        i = i.replace('.','')
        i = i.replace(':','')
        i = i.replace('\xc3\xa8','e')
        moremovies2.append(i)          

In [24]:
#combining worst and best movies to moremovies2

for i in worstmovies:
    moremovies2.append(i)

In [25]:
#obtaining movie info data from omdbapi

import json
import urllib2

url_template = "http://www.omdbapi.com/?t={}&plot=short&r=json"

results = []
errors = []

for i in set(moremovies2):
    i = i.replace(" ","+")
    a = json.load(urllib2.urlopen(url_template.format(i)))
    if a['Response'] == "True" and a['Type'] != 'series':
        results.append(a)

df = pd.DataFrame(results)

In [29]:
url_template = "http://www.omdbapi.com/?t={}&plot=short&r=json"

results = []
errors = []

for i in set(worstmovies):
    i = i.replace(" ","+")
    a = json.load(urllib2.urlopen(url_template.format(i)))
    if a['Response'] == "True" and a['Type'] != 'series':
        results.append(a)

dfworst = pd.DataFrame(results)

In [31]:
dfworst.head(20)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,"Alice Brown, Eric Colvin, Jonathan Coote, Eamo...",N/A,UK,Mark Forbes,"Short, Horror",English,N/A,A business exec breaks down in his car in rura...,N/A,N/A,19 Oct 2000,True,20 min,Hobgoblin,movie,Norman Adams (screenplay),2000,tt1863272,N/A,N/A
1,"Rajesh Khanna, Tanuja, Moushumi Chatterjee, Ra...",N/A,India,Jambulingam,"Thriller, Mystery, Family",Hindi,N/A,"Ram comes from a poor family, and works as a l...",N/A,N/A,N/A,True,N/A,Humshakal,movie,"Inder Raj Anand (dialogue), Poovai Krishnan (s...",1974,tt0298366,5.5,22
2,"Steve Reeves, Giacomo Rossi Stuart, Carla Marl...",N/A,"Italy, France, Yugoslavia",Giorgio Venturini,"Adventure, Drama",Italian,N/A,Aeneas leads escapees from the Trojan war to n...,http://ia.media-imdb.com/images/M/MV5BMTY0NjM1...,N/A,01 Jun 1964,True,95 min,The Avenger,movie,"Publius Vergilius Maro (epic poem: ""Aeneis""), ...",1962,tt0056174,5.2,270
3,"Philip Morton, June Travis, George Perry, Lois...",N/A,USA,"Bill Rebane, Herschell Gordon Lewis","Sci-Fi, Horror",English,N/A,"A space capsule crash-lands on Earth, and the ...",http://ia.media-imdb.com/images/M/MV5BMTg1NTIz...,TV-PG,01 Jul 1965,True,70 min,Monster a-Go Go,movie,"Jeff Smith (screenplay), Dok Stanford (screenp...",1965,tt0059464,2.3,"6,177"
4,"Michael Keaton, Jack Nicholson, Kim Basinger, ...",Won 1 Oscar. Another 9 wins & 22 nominations.,"USA, UK",Tim Burton,"Action, Adventure","English, French",69,The Dark Knight of Gotham City begins his war ...,http://ia.media-imdb.com/images/M/MV5BMTYwNjAy...,PG-13,23 Jun 1989,True,126 min,Batman,movie,"Bob Kane (Batman characters), Sam Hamm (story)...",1989,tt0096895,7.6,"266,989"
5,"Sudeep, Nani, Samantha Ruth Prabhu, Hamsa Nandini",33 wins & 25 nominations.,India,"S.S. Rajamouli, J.V.V. Sathyanarayana","Comedy, Drama, Fantasy","Telugu, Hindi, Kannada, Thai, Portuguese, Japa...",N/A,A murdered man is reincarnated as a housefly a...,http://ia.media-imdb.com/images/M/MV5BMTU2NDIx...,N/A,05 Jul 2012,True,145 min,Eega,movie,"S.S. Kanchi (story treatment), Janardhan Mahar...",2012,tt2258337,8.0,"11,992"
6,"Charlie Sheen, Hilary Duff, Eva Longoria, Wayn...",N/A,USA,Lawrence Kasanoff,"Animation, Action, Comedy",English,N/A,The evil Brand X joins a supermarket that beco...,http://ia.media-imdb.com/images/M/MV5BMjMzNzI3...,PG,12 Feb 2013,True,91 min,Foodfight!,movie,"Lawrence Kasanoff (story), Joshua Wexler (stor...",2012,tt0249516,1.7,"4,214"
7,"Patrick Frost, Cale Morgan",N/A,Australia,David De Vries,"Short, Thriller",English,N/A,"Morgan, a hit man, takes his mentor Hannibal i...",N/A,N/A,30 Jun 2012,True,N/A,Parting Shot,movie,David De Vries,2012,tt2380994,N/A,N/A
8,"Jon Voight, Scott Baio, Vanessa Angel, Skyler ...",5 nominations.,"Germany, UK, USA",Bob Clark,"Comedy, Family, Sci-Fi",English,9,A group of smart-talking toddlers find themsel...,http://ia.media-imdb.com/images/M/MV5BNjY4NjM3...,PG,27 Aug 2004,True,88 min,Superbabies: Baby Geniuses 2,movie,"Steven Paul (story), Gregory Poppen (screenplay)",2004,tt0270846,1.9,"24,996"
9,"Kurt Russell, Wilford Brimley, T.K. Carter, Da...",3 nominations.,USA,John Carpenter,"Horror, Sci-Fi","English, Norwegian",N/A,It's the first week of winter in 1982. An Amer...,http://ia.media-imdb.com/images/M/MV5BNTQ5ODU0...,R,25 Jun 1982,True,109 min,The Thing,movie,"Bill Lancaster (screenplay), John W. Campbell ...",1982,tt0084787,8.2,"255,717"


In [26]:
#deleting unnecessary columns

del df['Plot']
del df['Poster']
del df['Writer']
del df['imdbID']
del df['Released']
del df['Response']
del df['Type']
del df['Actors']

In [27]:
df.drop_duplicates()
df.head(100)

,Awards,Country,Director,Genre,Language,Metascore,Rated,Runtime,Title,Year,imdbRating,imdbVotes
0,Won 1 Oscar. Another 1 win.,USA,"Ben Hecht, Charles MacArthur",Drama,English,N/A,APPROVED,76 min,The Scoundrel,1935,6.8,162
1,Nominated for 2 Oscars. Another 10 wins & 1 no...,Japan,Hiroshi Teshigahara,"Drama, Thriller",Japanese,N/A,NOT RATED,123 min,Woman in the Dunes,1964,8.5,"11,094"
2,N/A,India,Jambulingam,"Thriller, Mystery, Family",Hindi,N/A,N/A,N/A,Humshakal,1974,5.5,22
3,Won 1 Oscar. Another 2 wins & 2 nominations.,USA,Alfred E. Green,"Biography, Drama, History",English,N/A,PASSED,90 min,Disraeli,1929,6.5,710
4,Won 1 Oscar. Another 10 wins & 14 nominations.,USA,Peter Yates,"Comedy, Drama, Sport","English, Italian, French",N/A,PG,101 min,Breaking Away,1979,7.7,"17,324"
5,N/A,USA,David Licata,"Comedy, Short",English,N/A,N/A,9 min,8 1/2 x 11,1999,6.5,15
6,1 win & 1 nomination.,Japan,Kenji Mizoguchi,Drama,Japanese,N/A,NOT RATED,124 min,Sansho the Bailiff,1954,8.4,"10,122"
7,Won 4 Oscars. Another 4 wins & 18 nominations.,USA,Joshua Logan,"Drama, Romance","English, Japanese",N/A,UNRATED,147 min,Sayonara,1957,7.3,"4,235"
8,3 wins & 7 nominations.,Canada,David Cronenberg,"Horror, Mystery, Sci-Fi",English,60,R,87 min,Videodrome,1983,7.3,"59,565"
9,Nominated for 1 Oscar. Another 9 wins & 3 nomi...,Japan,Akira Kurosawa,"Crime, Drama",Japanese,N/A,UNRATED,88 min,Rashomon,1950,8.3,"101,512"


In [28]:
len(df)

964

In [ ]:
#separating genres for movies with more than one listed. 

def genrecounts(value):
    value.split(', ')
    Drama,Romance,Comedy,Thriller,Biography,Adventure,Mystery,Crime,Documentary,History,SciFi,FilmNoir,Action = 0,0,0,0,0,0,0,0,0,0,0,0,0
    if "Drama" in value:
        Drama +=1
    if "Romance" in value:
        Romance +=1
    if "Comedy" in value:
        Comedy +=1
    if "Thriller" in value:
        Thriller +=1
    if "Biography" in value:
        Biography +=1
    if "Adventure" in value:
        Adventure +=1
    if "Mystery" in value:
        Mystery +=1
    if "Crime" in value:
        Crime +=1
    if "Documentary" in value:
        Documentary +=1
    if "History" in value:
        History +=1
    if "Sci-Fi" in value:
        SciFi +=1
    if "Film-Noir" in value:
        FilmNoir +=1
    if "Action" in value:
        Action +=1
    return Drama,Romance,Comedy,Thriller,Biography,Adventure,Mystery,Crime,Documentary,History,SciFi,FilmNoir,Action

dfgenrescount = df.Genre.to_frame().applymap(genrecounts)

In [ ]:
#adding dfgenrescount to dataframe

dfgenres = pd.DataFrame(columns = ["Drama","Romance","Comedy","Thriller","Biography","Adventure","Mystery","Crime","Documentary","History","SciFi","FilmNoir","Action"])
for i in range(len(dfgenrescount)):
    dfgenres.loc[i] = [x for x in dfgenrescount.Genre[i]]
dfgenres.head()

In [ ]:
#creating 

genrelist2 = []
genrelist1 = ["Drama","Romance","Comedy","Thriller","Biography","Adventure","Mystery","Crime","Documentary","History","SciFi","FilmNoir","Action"]
for i in genrelist1:
    genrelist2.append(np.sum([dfgenres[i]]))

In [ ]:
#creating keys for merging the dataframes

df['key'] = [x for x in range(len(df))]
dfgenres['key'] = [x for x in range(len(dfgenres))]

In [ ]:
#merging genres df with main df

df = pd.merge(df,dfgenres,on = 'key')
del df['key']
df.head(2)

In [ ]:
#cleaning runtime by removing 'min'

df['Runtime'] = df['Runtime'].apply(lambda x: int(x[:len(x)-4])

In [ ]:
df.head(2)

In [ ]:
#cleaning country column by extracting only first country listed

df['Country'] = df.Country.apply(lambda x: x.split(',')[0])
df['Country'] = df.Country.apply(lambda x: "Germany" if x == "West Germany" else x)

In [ ]:
#plotting the number of movies per country

df.Country.value_counts().plot(kind = 'bar', figsize = (15,6))

In [ ]:
#adding dummies for countries

countrylist = ['USA','UK','France','Italy','Japan','Germany','Sweden','Canada','Spain','China','Australia']
df['CountryTop']= df.Country.apply(lambda x: x if x in countrylist else 'Other')
df['key'] = range(len(df['CountryTop']))
df = pd.concat([df.drop('key',axis=1),pd.get_dummies(df['CountryTop'])], axis = 1)
del df['CountryTop']
df.head()

In [ ]:
#converting language to English/Not English

df['LangEnglish'] = df.Language.apply(lambda x: 1 if "English" in x else 0)

In [ ]:
#converting awards to containing 'Oscar' or not.

df['Awards'] = df.Awards.apply(lambda x: 1 if "Oscar" in x else 0)

In [ ]:
df.head()

In [ ]:
#removing unnecessary columns

del df['Metascore']
del df['Country']
del df['Director']
del df['Language']
del df['Genre']

In [ ]:
#creating dummies for ratings

ratings = ['R','PG','PG-13','G','X','M','NC-17']
df['Rated'] = df.Rated.apply(lambda x: x if x in ratings else "Other")
df['key'] = range(len(df['Rated']))
df = pd.concat([df.drop('key',axis=1),pd.get_dummies(df['Rated'])], axis = 1)
del df['Rated']
df.head()

In [ ]:
df.set_index('Title')

In [ ]:
#cleaning imdbVotes be removing commas and turning strings to np.nan

df['imdbVotes'] = df.imdbVotes.apply(lambda x: x.replace(',',''))
df['imdbVotes'] = df.imdbVotes.apply(lambda x: x if x != "N/A" else np.nan)

In [ ]:
#drop np.nan from imdbRating

df['imdbRating'] = df.imdbRating.apply(lambda x: x if x != "N/A" else np.nan)
df = df.dropna(subset=['imdbRating'], how='all')
len(df)

In [ ]:
#finding string, converting to np.nan and removing

df = df.replace({"N/":np.nan})
df = df.dropna(subset=['Runtime'], how='all')
df = df.dropna(subset=['imdbVotes'], how='all')

In [ ]:
#sns.pairplot(df[[, hue="species")

In [ ]:
#setting X and y

X = df[['Awards','Runtime', 'Year','imdbVotes','Drama','Romance', 'Comedy','Thriller', 
       'Biography',   u'Adventure', 'Mystery', 'Crime', 'Documentary', 'History', 'SciFi', 'FilmNoir', 
       'Action', 'Australia', 'Canada', 'China','France', u'Germany', 'Italy', 'Japan', 'Other', 'Spain', 'Sweden',
       'UK','USA', 'LangEnglish', 'G', 'M','NC-17', 'Other', 'PG', 'PG-13', 'R','X']]
y = df.imdbRating

In [ ]:
#setting train and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=42)

In [ ]:
#decisiontree regressor model

from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import KFold, cross_val_score
cv = KFold(len(y), shuffle=True)
dt = DecisionTreeRegressor(random_state=1)
dt.fit(X_train,y_train)
dtpreds = dt.predict(y_test)
dtscores = cross_val_score(dt, X, y, cv=cv)
dtscores.mean()

In [ ]:
#randomforest regressor model

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train,y_train)
rfpreds = rf.predict(y_test)
rfscores = cross_val_score(rf, X, y, cv=cv)
rfscores.mean()

In [ ]:
#adaboost regressor model

from sklearn.ensemble import AdaBoostRegressor
ab = AdaBoostRegressor()
ab.fit(X_train,y_train)
abpreds = ab.predict(y_test)
abscores = cross_val_score(ab, X, y, cv=cv)
abscores.mean()

In [ ]:
#gradient boost regressor model

from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
gb.fit(X_train,y_train)
gbpreds = gb.predict(y_test)
gbscores = cross_val_score(gb, X, y, cv=cv)
gbscores.mean()

In [ ]:
#plotting predicted valies against actuals

plt.plot(preds)
plt.plot(y_test.values)

In [ ]:
def tree_plot(model, m=None):
    plt.scatter(y_test, gb_ypred, color='darkred', alpha=.7)
    m,b = np.polyfit(y_test,gb_ypred,1)
    plt.plot(y_test,(m*y_test+b), linewidth=1, color='black')
    plt.text(7.5,5.7, "CV_Score: "+str(m), fontsize=15, color='darkblue')
    plt.suptitle('Gradient Boost IMDB Model Regressor', fontsize=20)
    plt.xlabel('Y Test Values', fontsize=15)
    plt.ylabel('Y Pred values', fontsize=15)

tree_plot(gb, gb_scores)